In [1]:
import sys
!{sys.executable} -m pip install PyPDF2
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install nltk

In [12]:
from pyspark import SparkContext, SparkFiles
from pathlib import Path
from bs4 import BeautifulSoup
from collections import Counter
from operator import add
import os
import requests
import PyPDF2
import re
import nltk
import timeit
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
def get_books_urls(url):
    urlsWithName = []
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    for a in soup.find_all("a", href=True):
        arr = a["href"].split(".")
        if arr[-1] == "pdf":
            urlsWithName.append(("http:" +a["href"],a.string))
    return urlsWithName

In [4]:
def get_books_and_create_folder(urls, target):
    if not os.path.isdir(target):
        os.makedirs(target)
    for url in urls:
        file = Path(target,url[1] + ".pdf")
        if not file.exists ():
            response = requests.get(url[0])
            with open(Path(target, a.string + ".pdf"), "wb+") as f:
                f.write(response.content)
        else:
            print ("File already exists")


In [5]:
def save_text_from_pdf(name):
    file = Path(name + ".txt")
    if not file.exists ():
        fileReader = PyPDF2.PdfFileReader(name)
        with open(Path(name + ".txt"), "a+") as f:
            for page in fileReader.pages:
                f.write(page.extractText())

In [6]:
folderName = 'freud'
urls = get_books_urls("https://holybooks.com/sigmund-freud-the-complete-works/")
get_books_and_create_folder(urls, folderName)

File already exists


In [7]:
p = Path(folderName)
pdfFiles = list(p.glob('*.pdf'))
for b_p in pdfFiles:
    save_text_from_pdf(os.path.join(folderName, b_p.name))


In [8]:
sc = SparkContext("local", "first app")

In [9]:
p = Path(folderName)
allTextFiles = list(p.glob('*.txt'))
files = []
for b_p in allTextFiles:
    files.append(sc.textFile(os.path.join(folderName, b_p.name)).cache())

In [10]:
def is_noun(word : str) -> bool:
    if "s" == word or "i" == word:
        return False
    partOfSpeech = nltk.pos_tag([word])
    if "NN" == partOfSpeech[0][1]:
        return True
    return False

In [13]:
start = timeit.default_timer()
for file in files:
    lowerCaseText = file.map(lambda x: x.lower())
    words = lowerCaseText.map(lambda x : re.findall(r"\b[^\d\W]+\b", x)).flatMap(lambda x: x)
   
    wordsForCounting = words.map(lambda x: (x,1))
    countedNouns = wordsForCounting.reduceByKey(add).filter(lambda x: is_noun(x[0])).collect()
    
    kNouns = Counter(dict(countedNouns))
    kWords = Counter(words.collect())
    
    print(words.count())
    print(kWords.most_common(10))
    print(kNouns.most_common(3))
end = timeit.default_timer()
print(end-start)

2068906
[('the', 142371), ('of', 94924), ('to', 60988), ('in', 50950), ('a', 48808), ('and', 46208), ('that', 33805), ('is', 30431), ('it', 27788), ('which', 22649)]
[('dream', 6115), ('analysis', 3584), ('time', 3456)]
15.255547217999265
